<a href="https://colab.research.google.com/github/Kendi7/AI-for-SE-WEEK-3/blob/NLPamazon-reviws/Nlpamazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 📦 Install dependencies (run once)
!pip install -q kaggle spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import files

# Upload your kaggle.json file
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [3]:
import os
import shutil

# Create the directory and move file
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Set permissions
os.chmod("/root/.kaggle/kaggle.json", 600)


In [5]:
import os
import shutil

# Step 1: Set environment variable
os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"

# Step 2: Also copy to default config path for redundancy
os.makedirs("/root/.config/kaggle", exist_ok=True)
shutil.copy("/root/.kaggle/kaggle.json", "/root/.config/kaggle/kaggle.json")


'/root/.config/kaggle/kaggle.json'

In [8]:
import os
import zipfile
import bz2
import spacy
from kaggle.api.kaggle_api_extended import KaggleApi

# 1. DOWNLOAD & UNZIP DATA
def download_amazon_reviews(slug="bittlingmayer/amazonreviews", out_dir="amazon_data"):
    api = KaggleApi()
    api.authenticate()
    os.makedirs(out_dir, exist_ok=True)

    # Download the zipped dataset
    api.dataset_download_files(slug, path=out_dir, unzip=True)
    print("✅ Dataset downloaded and unzipped.")

# 2. LOAD .BZ2 TEXT REVIEWS
def load_fasttext_samples(bz2_path, n=10):
    samples = []
    with bz2.open(bz2_path, "rt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= n: break
            label, text = line.split(maxsplit=1)
            samples.append(text.strip())
    return samples

# 3. DO NER + SIMPLE RULE-BASED SENTIMENT
def analyze_texts(texts):
    nlp = spacy.load("en_core_web_sm")
    pos_kw = ["love", "great", "amazing", "fantastic", "good", "excellent", "best", "perfect"]
    neg_kw = ["bad", "poor", "terrible", "disappointed", "worst", "broke", "slow", "awful"]

    results = []
    for txt in texts:
        doc = nlp(txt)
        ents = [ent.text for ent in doc.ents if ent.label_ in ("ORG", "PRODUCT")]

        sentiment = "Neutral"
        lower = txt.lower()
        if any(word in lower for word in pos_kw):
            sentiment = "Positive"
        elif any(word in lower for word in neg_kw):
            sentiment = "Negative"

        results.append({
            "text": txt,
            "entities": ents,
            "sentiment": sentiment
        })
    return results

# 4. ETHICS NOTES
def ethics_notes():
    print("""
🧭 Ethical Considerations:

1. ⚠️ Dataset Bias:
   - Overrepresentation of certain products, languages, or sentiment tones.
   - Some products may receive more feedback simply due to popularity.

2. ⚠️ Sentiment Ambiguity:
   - A 1-star review might contain sarcastic praise ("Great job breaking it!").

3. ⚠️ Prejudice in rules:
   - Simple keyword-based sentiment may misclassify nuanced texts.
   - Words like "slow" may be negative in tech reviews but neutral in novels.

✅ Mitigation Measures:

- Use spaCy's rule-based system for transparency and tunability.
- Apply `TensorFlow Fairness Indicators` when training ML models to track bias across groups (e.g., by product category).
- Combine rules with lightweight, domain-adapted ML sentiment classifiers.
- Collect user feedback and continuously audit misclassifications.
""")

# 5. MAIN
def main():
    data_dir = "amazon_data"
    download_amazon_reviews(out_dir=data_dir)

    # Locate .bz2 review file (usually under amazon_data/train.ft.txt.bz2)
    bz2_file = os.path.join(data_dir, "train.ft.txt.bz2")
    if not os.path.exists(bz2_file):
        for root, dirs, files in os.walk(data_dir):
            for f in files:
                if f.endswith(".bz2"):
                    bz2_file = os.path.join(root, f)
                    break

    print(f"📥 Loading reviews from: {bz2_file}")
    samples = load_fasttext_samples(bz2_file, n=10)

    print("🧠 Analyzing samples...\n")
    results = analyze_texts(samples)
    for i, r in enumerate(results, 1):
        print(f"--- Review {i} ---")
        print("Text     :", r["text"])
        print("Entities :", r["entities"])
        print("Sentiment:", r["sentiment"])
        print()

    ethics_notes()

# 🔃 Execute
main()


Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
✅ Dataset downloaded and unzipped.
📥 Loading reviews from: amazon_data/train.ft.txt.bz2
🧠 Analyzing samples...

--- Review 1 ---
Text     : Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
Entities : ['Chrono Cross']
Sentiment: Positive

--- Review 2 ---
Text     : The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to 